# Hello Nicolas!  
 The purpose of this code is to automatically produce dynamic plots of your raw data from the Karthala microscope.

Step 1: put all your recordings in a local folder on your computer.  Copy the path to 'master_folder_path' in the next cell.  Make sure to use forward slashes '/', not '\'.

Step 2: Execute the entire script.  (there will be an error, but that's on purpose) 

Step 3: in the master folder, you will find a excel file called table_empty.  Edit the last two columns ("Roi_skip" and "Roi_grouping"), and save as "table_done.xlsx" in the same folder.

Step 4: Execute the entire script again.  This time there should not be an error.  The plots should be produced in the "output" folder.    

In [1]:
master_folder_path = 'C:/Users/huynh/INT/nicolas_master_folder'

In [2]:
### Create output folder
import numpy as np
import os
plot_folder_path = os.path.join(master_folder_path, "output")

# Check if the output folder exists
if not os.path.exists(plot_folder_path):
    try:
        # Create the output folder if it doesn't exist
        os.makedirs(plot_folder_path)
        print(f"Folder 'output' created in {master_folder_path}")
    except OSError as e:
        print(f"Error: {e}")
else:
    print(f"Folder 'output' already exists in {master_folder_path}")

Folder 'output' already exists in C:/Users/huynh/INT/nicolas_master_folder


In [3]:
import pandas as pd

master_folder_name = master_folder_path.split('/')[-1]

def get_file_paths(folder_path, max_depth=4):
    tdms_files = []
    xls_files = []
    
    # Recursive function to search for .tdms and .xls files
    def search_files(current_path, depth):
        if depth <= max_depth:
            for item in os.listdir(current_path):
                item_path = os.path.join(current_path, item)
                if os.path.isdir(item_path):
                    search_files(item_path, depth + 1)
                elif os.path.isfile(item_path):
                    if item.endswith('pmt1.tdms') or item.endswith('.xls'):
                        if depth == 2 and item == 'points':
                            # Skip the 'points' folder at depth 2
                            continue
                        if item.endswith('pmt1.tdms'):
                            tdms_files.append(item_path)
                        elif item.endswith('.xls'):
                            xls_files.append(item_path)
    
    search_files(folder_path, 1)
    return tdms_files, xls_files

file_paths = get_file_paths(master_folder_path)
tdms_files, xls_files = get_file_paths(master_folder_path)

if len(tdms_files) == len(xls_files):
    n_files = len(tdms_files)
else:
    raise IndexError(f"number of tdms doesn't match number of xls")

print(n_files, "TDMS files found:")
for file_path in tdms_files:
    print(file_path)
# print("\n")
# print(n_xls_files, "XLS files found:")
# for file_path in xls_files:
#     print(file_path)


13 TDMS files found:
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0001\points0001_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0002\points0002_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0003\points0003_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0004\points0004_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0005\points0005_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0006\points0006_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0007\points0007_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0008_best\points0008_pmt1.tdms
C:/Users/huynh/INT/nicolas_master_folder\2022_12_02P8_jedi_JC_best\points\points0009\points0009_pmt1.tdms
C:/Users/huynh/INT/n

In [4]:
import re

def get_plot_name(file_path, master_folder_name):
    # print(file_path)
    words = re.split(r'[\\\/]', file_path)
    start_index = words.index(master_folder_name)
    name = str(words[start_index+1])  + "--" + str(words[start_index+3]) + '.html'
    return name

def add_trial_prefix(names):
    return [f"trial_{idx}--{name}" for idx, name in enumerate(names, start=1)]    

# get_plot_name(test,master_folder_name)


In [5]:
from nptdms import TdmsFile

tdms_files = file_paths[0]
n_files = len(tdms_files)

all_traces = []
all_roi_counts = []
file_names = []

for file in  range(n_files):

    tdms_file = TdmsFile.read(tdms_files[file])
    file_name = get_plot_name(tdms_files[file],master_folder_name)
    file_names.append(file_name)

    parameter_group = tdms_file['parameter']
    num_channel = parameter_group['value']
    num = num_channel[:]
    numpoint = int(num[8]) # 8 is the array representing the number of points

    all_roi_counts.append(numpoint)

    traces = []
    for point in range(numpoint):
        traces.append(tdms_file['PMT1']['POI ' + str(point) + ' '].data)
    all_traces.append(traces)

file_names = add_trial_prefix(file_names)

In [6]:
### EXPORT TABLE TO BE COMPLETED
df_empty = pd.DataFrame({
    'Point': [point for point, count in zip(file_names, all_roi_counts) for _ in range(count)],
    'Roi': [j for count in all_roi_counts for j in range(count)],
    'Roi_skip': ['0' for _ in range(sum(all_roi_counts))],
    'Roi_grouping (use similar integers to indicate grouping)': ['0' for _ in range(sum(all_roi_counts))]
})

# Reset index values at point
df_empty['Roi'] = df_empty.groupby('Point').cumcount()

# print(df_empty)

# Specify the file path
table_empty_name = 'table_empty.xlsx'
table_file_path = os.path.join(master_folder_path, table_empty_name) 

# Write DataFrame to Excel
df_empty.to_excel(table_file_path, index=False)

print("Excel file created successfully.")

Excel file created successfully.


In [7]:
### IMPORT TABLE
table_done_name = 'table_done.xlsx'
table_file_path = os.path.join(master_folder_path, table_done_name) 

# Read Excel file into DataFrame
df_done = pd.read_excel(table_file_path)

point_column = df_done.iloc[:, 0].values
roi_skip_column = (df_done.iloc[:, 2].values).astype(int)
roi_grouping_column = (df_done.iloc[:, 3].values).astype(int)

roi_grouping = {}
roi_skip = {}

for a_val, b_val in zip(point_column, roi_grouping_column):
    if a_val not in roi_grouping:
        roi_grouping[a_val] = []
    roi_grouping[a_val].append(b_val)
roi_grouping = [np.array(values) for values in roi_grouping.values()]


### Use skip column to empty frames
for a_val, b_val in zip(point_column, roi_skip_column):
    if a_val not in roi_skip:
        roi_skip[a_val] = []
    roi_skip[a_val].append(b_val)
roi_skip = [np.array(values) for values in roi_skip.values()]

for i in range(n_files):
    for j in range(len(all_traces[i])):
        # Check if the corresponding element in list_of_arrays2 is zero
        if roi_skip[i][j] != 0:
            # If it's not zero, replace the element in list_of_arrays1 with an empty array
            all_traces[i][j] = np.array([])

# print(roi_grouping)
# print(roi_skip)

In [8]:
def sum_arrays(arrays):
    max_length = max(len(arr) for arr in arrays)
    summed_array = np.zeros(max_length)
    for arr in arrays:
        summed_array[:len(arr)] += arr
    return summed_array

In [9]:
import plotly.graph_objects as go

for trial in range(n_files):

    fig = go.Figure()
    for i in range(all_roi_counts[trial]):
        fig.add_trace(go.Scatter(y=all_traces[trial][i], mode='lines', name=f'roi {i}'))

    time_series = all_traces[trial]
    grouping = roi_grouping[trial]
    summed_time_series = []
    summed_indices = []

    for group_val in np.unique(grouping):
        # Filter the time series array based on the current group value
        group_time_series = [ts_array for ts_array, group in zip(time_series, grouping) if group == group_val]
        group_indices = [idx for idx, group in enumerate(grouping) if group == group_val]
        
        # Skip the group if no corresponding time series or only one time series are found
        if not group_time_series or len(group_time_series) == 1:
            continue
        
        # Sum the time series arrays within the same group element-wise
        summed_array = sum_arrays(group_time_series)
        
        # Append the summed array and corresponding indices to the lists
        summed_time_series.append(summed_array)
        summed_indices.append(group_indices)

    for i in range(len(summed_time_series)):
        fig.add_trace(go.Scatter(y=summed_time_series[i], mode='lines', name=f'rois {[summed_indices[i]][0]}'))

    fig.update_layout(
        title=(f'Plot : {file_names[trial]}'),
        xaxis_title="Time (frame)",
        yaxis_title="Amplitude"
    )

    # Save the plot as an HTML file in the output folder
    output_path = os.path.join(plot_folder_path, file_names[trial])
    fig.write_html(output_path)
    print(f"Plot saved at: {output_path}")

Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_1--2022_12_02P8_jedi_JC_best--points0001.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_2--2022_12_02P8_jedi_JC_best--points0002.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_3--2022_12_02P8_jedi_JC_best--points0003.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_4--2022_12_02P8_jedi_JC_best--points0004.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_5--2022_12_02P8_jedi_JC_best--points0005.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_6--2022_12_02P8_jedi_JC_best--points0006.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_7--2022_12_02P8_jedi_JC_best--points0007.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_8--2022_12_02P8_jedi_JC_best--points0008_best.html
Plot saved at: C:/Users/huynh/INT/nicolas_master_folder\output\trial_9--202